In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

from pydrake.geometry.optimization import HPolyhedron
from pydrake.trajectories import PiecewisePolynomial

from spp.bezier import BezierSPP
from spp.linear import LinearSPP

In [ ]:
from scipy.spatial import ConvexHull, HalfspaceIntersection

def vertices(P):
    halfspaces = np.column_stack((P.A(), -P.b()))
    P = HalfspaceIntersection(halfspaces, P.ChebyshevCenter())
    return P.intersections

def plot_hpolytope(P, **kwargs):
    V = vertices(P)
    hull = ConvexHull(V) # orders vertices counterclockwise
    plt.fill(*V[hull.vertices].T, **kwargs)

In [ ]:
x_start = np.array([-.1, 0])
x_goal = np.array([1.1, 0])
relaxation = False

sr2 = np.sqrt(2)

A1 = np.array([[1, 0], [0, 1], [-1, 0], [0, -1]])
b1 = np.array([0, 2, 1, 0])

A2 = np.array([[1, 0], [1, 1], [0, 1], [-1, 0], [0, -1]])
b2 = np.array([.5+1/sr2, 0, 0, 1, 2])

A3 = np.array([[1, 0], [0, 1], [-1, 0], [0, -1]])
b3 = np.array([1, 2, 0, -1])

A4 = np.array([[1, 0], [0, 1], [-1, 1], [-1, 0], [0, -1]])
b4 = np.array([2.5, -1/sr2, -1-2/sr2, -.5-1/sr2, 2])

A5 = np.array([[1, 0], [0, 1], [-1, 0], [-1, -1], [0, -1]])
b5 = np.array([2.5, 2, -1, -1, 1/sr2])

regions = [
    HPolyhedron(A1, b1),
    HPolyhedron(A2, b2),
    HPolyhedron(A3, b3),
    HPolyhedron(A4, b4),
    HPolyhedron(A5, b5),
]

plt.figure()

for P in regions:
    plot_hpolytope(P, fc='lightcyan', ec='k')

plt.plot(x_start[0], x_start[1], 'kx')
plt.plot(x_goal[0], x_goal[1], 'kx')
plt.fill(
    [1, 1, 0, 0,  .5+1/sr2, 1+1/sr2],
    [0, 1, 1, 0, -.5-1/sr2,  -1/sr2],
    fc='lightcoral',
    ec='k'
)
plt.axis('square')
plt.xlim([-1, 2.5])
plt.ylim([-2, 2])

l_spp = LinearSPP(regions)
waypoints, _, _ = l_spp.SolvePath(x_start, x_goal, relaxation)
t_knots = np.linspace(0, 1,  waypoints.shape[1])
l_traj = PiecewisePolynomial.FirstOrderHold(t_knots, waypoints)

n_samples = 500
l_times = np.linspace(
    l_traj.start_time(),
    l_traj.end_time(),
    n_samples
)
values = np.squeeze([l_traj.value(t) for t in l_times])
plt.plot(*values.T, 'g-')

b_spp = BezierSPP(regions, order=3, continuity=1)
b_spp.addTimeCost(1)
b_spp.addVelocityLimits([-1, -1], [1, 1])
b_traj, _, _ = b_spp.SolvePath(x_start, x_goal, relaxation)

b_times = np.linspace(
    b_traj.start_time(),
    b_traj.end_time(),
    n_samples
)
values = np.squeeze([b_traj.value(t) for t in b_times])
plt.plot(*values.T, 'b-')

In [ ]:
plt.figure()
b_velocity = np.array([b_traj.EvalDerivative(t) for t in b_times])
plt.plot(b_times, b_velocity[:, 0], label='Horizontal velocity')
plt.plot(b_times, b_velocity[:, 1], label='Vertical velocity')
plt.plot(b_times, np.ones(len(b_times)), 'r--', label='Velocity limit')
plt.plot(b_times, -np.ones(len(b_times)), 'r--')
plt.legend()